In [ ]:
# using SBERT as a classifier

In [ ]:
!pip install torch transformers memory_profiler datasets
import time
import datetime
tic = time.time()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 8.6 MB/s eta 0:00:00


In [ ]:
import torch
import random
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_metric
%load_ext memory_profiler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/MyDrive/ReportCC++/SBERT

/content/drive/MyDrive/ReportCC++/SBERT


In [ ]:
dfNew = pd.read_json("function.json")

In [ ]:
df = pd.DataFrame(dfNew, columns=['target', 'func'])

In [ ]:
df

In [ ]:
# df = pd.read_csv("bbc-text.csv")

In [ ]:
df

In [ ]:
def set_seed(seed: int):
  random.seed(seed)
  np.random.seed(seed)
  if is_torch_available():
      torch.manual_seed(seed)
      torch.cuda.manual_seed_all(seed)

  if is_tf_available():
      import tensorflow as tf

      tf.random.set_seed(seed)

set_seed(42)

In [ ]:
def TextClassification_with_Transformer(model_name: str, Data: pd.Series, Target:pd.Series, test_size: np.float64, max_length: int, num_labels: int, num_epochs: int, metrics_name: str):

  # Make data
  X = Data
  y = Target
  y = pd.factorize(y)[0]

  # Load Metrics
  metric = load_metric(metrics_name)

  # Split Data
  X_train, X_test, y_train, y_test = train_test_split(X.tolist(), y, test_size=test_size)

  # Call the Tokenizer
  tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=True)

  # Encode the text
  train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=max_length)
  valid_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=max_length)



  class MakeTorchData(torch.utils.data.Dataset):
      def __init__(self, encodings, labels):
          self.encodings = encodings
          self.labels = labels

      def __getitem__(self, idx):
          item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
          item["labels"] = torch.tensor([self.labels[idx]])
          return item

      def __len__(self):
          return len(self.labels)

  # convert our tokenized data into a torch Dataset
  train_dataset = MakeTorchData(train_encodings, y_train.ravel())
  valid_dataset = MakeTorchData(valid_encodings, y_test.ravel())


  # Call Model
  model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = num_labels).to("cuda")

  # Create Metrics
  def compute_metrics(eval_pred):

    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    # 'micro', 'macro', etc. are for multi-label classification. If you are running a binary classification, leave it as default or specify "binary" for average
    #return metric.compute(predictions=predictions, references=labels, average="micro")
    return metric.compute(predictions=predictions, references=labels)


  # Specifiy the arguments for the trainer
  training_args = TrainingArguments(
      output_dir='./results',          # output directory
      num_train_epochs=num_epochs,     # total number of training epochs
      per_device_train_batch_size=8,   # batch size per device during training
      per_device_eval_batch_size=20,   # batch size for evaluation
      warmup_steps=500,                # number of warmup steps for learning rate scheduler
      weight_decay=0.01,               # strength of weight decay
      logging_dir='./logs',            # directory for storing logs
      load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
      metric_for_best_model = metrics_name,    # select the base metrics
      logging_steps=200,               # log & save weights each logging_steps
      save_steps=200,
      evaluation_strategy="steps",     # evaluate each `logging_steps`
  )

  # Call the Trainer
  trainer = Trainer(
      model=model,                         # the instantiated Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=train_dataset,         # training dataset
      eval_dataset=valid_dataset,          # evaluation dataset
      compute_metrics=compute_metrics,     # the callback that computes metrics of interest
  )

  # Train the model
  trainer.train()

  # Call the summary
  trainer.evaluate()



  return trainer, model

In [ ]:
%%time
%%memit
sbert_trainer, sbert_model = TextClassification_with_Transformer(model_name = 'sentence-transformers/all-mpnet-base-v2',
                                                                 Data = df.func,
                                                                 Target = df.target,
                                                                 test_size = 0.10,
                                                                 max_length = 512,
                                                                 num_labels =2,
                                                                 num_epochs = 3,
                                                                 metrics_name='precision')